In [1]:
import cv2
import mediapipe as mp
import time

2023-06-06 17:23:13.604077: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 17:23:13.708960: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-11.7/lib64:
2023-06-06 17:23:13.708979: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-06 17:23:14.312317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not 

In [2]:
def get_bounding_box(hand_landmarks, image_width, image_height):
    x_min = image_width
    y_min = image_height
    x_max = 0
    y_max = 0

    for landmark in hand_landmarks.landmark:
        x = int(landmark.x * image_width)
        y = int(landmark.y * image_height)

        if x < x_min:
            x_min = x
        if y < y_min:
            y_min = y
        if x > x_max:
            x_max = x
        if y > y_max:
            y_max = y

    # Add some padding to the bounding box
    padding = 20
    x_min -= padding
    y_min -= padding
    x_max += padding
    y_max += padding

    # Ensure the bounding box coordinates are within the image boundaries
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(image_width, x_max)
    y_max = min(image_height, y_max)

    return x_min, y_min, x_max, y_max

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [12]:

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 represents the default webcam, change it if necessary

# Check if the webcam is successfully opened
if not cap.isOpened():
    print("Unable to open the webcam.")
    exit()

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Counter for captured frames
frame_count = 0

n_frame = 0

mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

print("recording begins in 3 seconds")
time.sleep(3)
    

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Check if the frame is successfully read
    if not ret:
        print("Failed to read the frame.")
        break

    n_frame += 1
    # Flip the frame horizontally for a selfie view
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands in the frame
    results = hands.process(frame_rgb)
    # results = holistic_model.process(frame)

    # Check if hands are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Extract the bounding box coordinates for the hand
            x_min, y_min, x_max, y_max = get_bounding_box(hand_landmarks, frame.shape[1], frame.shape[0])

            # Crop the image around the hand region
            hand_image = frame[y_min:y_max, x_min:x_max]

            # Save the cropped hand image
            cv2.imwrite(f"hands/stay/2_{frame_count}.png", hand_image)

            # Increment the frame count
            frame_count += 1

        # Drawing Left hand Landmarks
        mp_drawing.draw_landmarks(
        frame,
        results.multi_hand_landmarks[0],
        mp_holistic.HAND_CONNECTIONS
        )

    # Display the frame in a window named 'Webcam Feed'
    cv2.imshow('Webcam Feed', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) == 27:  # 27 is the ASCII code for the Escape key
        break

    if n_frame > 300:
        print(f"{frame_count} frames captured.")
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()






recording begins in 3 seconds
312 frames captured.


In [ ]:
cv2.destroyAllWindows()
cap.release()

In [ ]:

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 represents the default webcam, change it if necessary

# Check if the webcam is successfully opened
if not cap.isOpened():
    print("Unable to open the webcam.")
    exit()

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Counter for captured frames
frame_count = 0

n_frame = 0

mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

print("recording begins in 3 seconds")
time.sleep(3)
    

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Check if the frame is successfully read
    if not ret:
        print("Failed to read the frame.")
        break

    n_frame += 1
    # Flip the frame horizontally for a selfie view
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands in the frame
    # results = hands.process(frame_rgb)
    results = holistic_model.process(frame)

    # Check if hands are detected
    if results.left_hand_landmarks:
        # Extract the bounding box coordinates for the hand
        x_min, y_min, x_max, y_max = get_bounding_box(results.left_hand_landmarks, frame.shape[1], frame.shape[0])

        # Crop the image around the hand region
        hand_image = frame[y_min:y_max, x_min:x_max]

        # Save the cropped hand image
        cv2.imwrite(f"hands/test/2_{frame_count}.png", hand_image)

        # Increment the frame count
        frame_count += 1

    # Drawing Right hand Landmarks
    mp_drawing.draw_landmarks(
      frame,
      results.left_hand_landmarks,
      mp_holistic.HAND_CONNECTIONS
    )
 


    # Display the frame in a window named 'Webcam Feed'
    cv2.imshow('Webcam Feed', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) == 27:  # 27 is the ASCII code for the Escape key
        break

    if n_frame > 300:
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()




